In [9]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Training only

In [12]:
import json
from data.data_set import Dataset
from model.inceptionv3_model import InceptionModel
from model.dummy_model import DummyModel

dataset_train = Dataset('/home/dschacherer_fme/input/csv_train_norm_cancer_small.csv', num_classes=2)
dataset_valid = Dataset('/home/dschacherer_fme/input/csv_valid_norm_cancer_small.csv', num_classes=2)
model = InceptionModel(num_classes=2)
#model = DummyModel()
print(model)

history = model.train(dataset_train, batch_size=4, epochs=2, output_path='/home/dschacherer_fme/output', validation_dataset=dataset_valid)

# save final model and history
model.save('/home/dschacherer_fme/output/trained_model')
with open('/home/dschacherer_fme/output/training_history.json', 'w') as hist:
    json.dump(history.history, hist)

# some prediction
data_point = dataset_valid.data_points[0]
prediction = model.make_prediction(data_point)
print('prediction on some sample: %f (reference is %d)' % (prediction, data_point.get_reference_value()))

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 512, 512, 3) 0                                            
__________________________________________________________________________________________________
conv2d_282 (Conv2D)             (None, 255, 255, 32) 864         input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_282 (BatchN (None, 255, 255, 32) 96          conv2d_282[0][0]                 
__________________________________________________________________________________________________
activation_282 (Activation)     (None, 255, 255, 32) 0           batch_normalization_282[0][0]    
____________________________________________________________________________________________